# TCOR

Term Co-ocurrence representation

## Definition
$$
w_{k, j} = tff(t_k, t_j) log(\frac{|T|}{T_k})
$$

$$
tff(t_k, t_j) = 
\begin{cases}
1 + log(\#(t_k, t_j)) &\quad if \ \#(t_k, t_j) > 0 \\
0 &\quad otherwise
\end{cases}
$$

Where:
- $T_k$ is the number of terms the k-th word co-occured with
- $|T|$ is the size of the vocabulary
- $\#(t_k, t_j)$ is the number of documents where $t_k$ co-occured with $t_j$
- $t$ is a word

## Import libraries

In [1]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import TweetTokenizer 
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import svm
%matplotlib inline
import torch
import nltk 

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


## Define utils

In [3]:
def get_texts_from_file(path_corpus, path_truth):
    txt = []
    y = []
    with open(path_corpus, "r") as f_corpus, open(path_truth, "r") as f_truth:
        for tuit in f_corpus:
            txt += [tuit]
        for label in f_truth:
            y += [label] 
    return txt, list(map(int, y))

In [4]:
def TCOR(tweets, V, dict_indices):
    def tff(tk, tj = None): # If one argument: return the number of words tk co-occurs with; if two, return the number of times they co-occur
        pass
    tcor = torch.zeros(len(V), len(V))
    for ik, k in enumerate(V):
        for ij, j in enumerate(V):
            #if ij == ik: continue
            tcor[ik, ij] = tff(tk, tj) * torch.log(len(V) / tff(tk))
    pass

## Set up corpus

In [5]:
tr_txt, tr_y = get_texts_from_file("../data/agresividad/mex_train.txt", "../data/agresividad/mex_train_labels.txt")
val_txt, val_y = get_texts_from_file("../data/agresividad/mex_val.txt", "../data/agresividad/mex_val_labels.txt")

In [36]:
corpus_palabras = []
corpus_tweets = []
tokenizer = TweetTokenizer()
for doc in tr_txt:
    corpus_palabras += tokenizer.tokenize(doc) # A single list
    corpus_tweets += [tokenizer.tokenize(doc)] # Several lists

In [45]:
vocab_size = 5000
fdist = nltk.FreqDist(corpus_palabras)
vocab = sorted([(fdist[key], key) for key in fdist])[:: -1][: vocab_size]
print(vocab[: 3])

[(3383, 'que'), (3357, 'de'), (2774, '.')]


In [8]:
indices = dict()
for i, w in enumerate(vocab):
    _, word = w
    indices[word] = i
print(len(indices))
list(indices)[:10]

5000


['que', 'de', '.', 'a', 'la', 'y', 'no', 'me', '!', 'el']

## Experiments

In [9]:
TCOR_tr = TCOR(tr_txt, vocab, indices)
TCOR_val = TCOR(val_txt, vocab, indices)
TCOR_tr.shape
TCOR_tr

AttributeError: 'NoneType' object has no attribute 'shape'

In [11]:
corpus_palabras.concordance("hola")

AttributeError: 'list' object has no attribute 'concordance'

In [41]:
from nltk.text import Text
corpus = Text(corpus_palabras)
corpusT = Text(corpus_tweets)

In [42]:
corpus.concordance('hola')
corpusT.concordance('hola')

Displaying 14 of 14 matches:
.. pontela poca madre ! ! ! @usuario hola toño ayer en tu programa como a las 
manda su rica verga 😍 😜 😝 😝 @usuario hola pinche toluca . te pones bien fría h
oger peor día pa chingar la momi . - hola pinche putita te quedaste sin mundia
mi prima la luchona . ella me dice " hola " y yo le contestó algo como " hola 
 hola " y yo le contestó algo como " hola prieta que quiere ser güera " venimo
as trabajan en el metro ! no mamar . hola pinche putita así que te gustan gran
a que los pario ooohhhooo oooohhoooo hola pinche putito te pones bien cachondo
r chupar y con la punta de mi lengua hola que bonito que por una parte ya te v
ue tu eres el del video que dice : " hola pinche putita " me quejaré toda la v
fue como hilo de media . mea culpa . hola tenia tiempo que no me conectaba . p
 ! depende de ustedes bebés @usuario hola a cual funcionario del gob del edome
ó mal … la neta el coral blanco y el hola pinche putita cachonda han sido lo m
cada loca que se tienen

In [16]:
corpus.collocations()

@usuario @usuario; puta madre; los putos; las putas; vale verga; todos
los; por qué; mamá luchona; sus putas; todas las; poca madre; putas
madres; mil putas; las mujeres; valer verga; esos putos; estoy hasta;
estoy loca; otra vez; creo que


In [26]:
corpus.common_contexts(['a', 'el'])

..._ver ._que que_otro "_que ._joto


In [35]:
corpus.concordance_list(['el'], lines = 10)

[ConcordanceLine(left=['conveniente', 'seria', 'que', 'lo', 'retes', 'a', 'unos', 'vergazos', 'mi', 'jelipe', '!', 'rómpele', 'la', 'madre', 'a', 'ese', 'pinchi', 'joto', '!'], query='el', right=['marica', 'de', 'mi', 'ex', 'me', 'tiene', 'bloqueada', 'de', 'todo', 'así', 'uno', 'no', 'puede', 'admirar', 'la', '"', 'belleza', '"'], offset=68, left_print=' rómpele la madre a ese pinchi joto !', right_print='marica de mi ex me tiene bloqueada de', line=' rómpele la madre a ese pinchi joto ! el marica de mi ex me tiene bloqueada de'),
 ConcordanceLine(left=['no', 'mejoran', '!', '!', '@usuario', '@usuario', 'y', 'si', 'este', '#hermano', '#hdp', '@usuario', 'tuyo', ';', 'se', 'queda', 'en', '#panamá', '?'], query='el', right=['🐷', 'd', '#javidu', 'sonreía', 'en', '#guatemala', 'estaría', 'mejor', 'allá', 'porque', 'lo', 'primero', 'que', 'busco', 'en', 'una', 'cuenta', 'es'], offset=339, left_print='@usuario tuyo ; se queda en #panamá ?', right_print='🐷 d #javidu sonreía en #guatemala est